In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl  # noqa: F401
    import dolfin  # noqa: F401
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl  # noqa: F401
        import dolfin  # noqa: F401

In [ ]:
try:
    import multiphenics
except ImportError:
    !python3 -m pip install git+https://github.com/multiphenics/multiphenics.git
    import multiphenics  # noqa: F401

In [ ]:
# Download data files
!mkdir -p data
![ -f data/circle_facet_region.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_facet_region.xml -O data/circle_facet_region.xml
![ -f data/circle_physical_region.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_physical_region.xml -O data/circle_physical_region.xml
![ -f data/circle.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle.xml -O data/circle.xml
!mkdir -p data/circle_restriction_left.rtc.xml
![ -f data/circle_restriction_left.rtc.xml/mesh_function_0.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_left.rtc.xml/mesh_function_0.xml -O data/circle_restriction_left.rtc.xml/mesh_function_0.xml
![ -f data/circle_restriction_left.rtc.xml/mesh_function_1.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_left.rtc.xml/mesh_function_1.xml -O data/circle_restriction_left.rtc.xml/mesh_function_1.xml
![ -f data/circle_restriction_left.rtc.xml/mesh_function_2.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_left.rtc.xml/mesh_function_2.xml -O data/circle_restriction_left.rtc.xml/mesh_function_2.xml
!mkdir -p data/circle_restriction_right.rtc.xml
![ -f data/circle_restriction_right.rtc.xml/mesh_function_0.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_right.rtc.xml/mesh_function_0.xml -O data/circle_restriction_right.rtc.xml/mesh_function_0.xml
![ -f data/circle_restriction_right.rtc.xml/mesh_function_1.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_right.rtc.xml/mesh_function_1.xml -O data/circle_restriction_right.rtc.xml/mesh_function_1.xml
![ -f data/circle_restriction_right.rtc.xml/mesh_function_2.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_right.rtc.xml/mesh_function_2.xml -O data/circle_restriction_right.rtc.xml/mesh_function_2.xml
!mkdir -p data/circle_restriction_interface.rtc.xml
![ -f data/circle_restriction_interface.rtc.xml/mesh_function_0.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_interface.rtc.xml/mesh_function_0.xml -O data/circle_restriction_interface.rtc.xml/mesh_function_0.xml
![ -f data/circle_restriction_interface.rtc.xml/mesh_function_1.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_interface.rtc.xml/mesh_function_1.xml -O data/circle_restriction_interface.rtc.xml/mesh_function_1.xml
![ -f data/circle_restriction_interface.rtc.xml/mesh_function_2.xml ] || wget https://github.com/multiphenics/multiphenics/raw/master/tutorials/03_lagrange_multipliers/data/circle_restriction_interface.rtc.xml/mesh_function_2.xml -O data/circle_restriction_interface.rtc.xml/mesh_function_2.xml

In [ ]:
from dolfin import *
from multiphenics import *

In [ ]:
mesh = Mesh("data/circle.xml")
subdomains = MeshFunction("size_t", mesh, "data/circle_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/circle_facet_region.xml")

In [ ]:
left = MeshRestriction(mesh, "data/circle_restriction_left.rtc.xml")
right = MeshRestriction(mesh, "data/circle_restriction_right.rtc.xml")
interface = MeshRestriction(mesh, "data/circle_restriction_interface.rtc.xml")

In [ ]:
V = FunctionSpace(mesh, "Lagrange", 2)
W = BlockFunctionSpace([V, V, V], restrict=[left, right, interface])

In [ ]:
u1u2l = BlockTrialFunction(W)
(u1, u2, l) = block_split(u1u2l)
v1v2m = BlockTestFunction(W)
(v1, v2, m) = block_split(v1v2m)

In [ ]:
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)
dS = dS(2)

In [ ]:
a = [[inner(grad(u1), grad(v1)) * dx(1), 0, l("-") * v1("-") * dS],
     [0, inner(grad(u2), grad(v2)) * dx(2), - l("+") * v2("+") * dS],
     [m("-") * u1("-") * dS, - m("+") * u2("+") * dS, 0]]
f = [v1 * dx(1), v2 * dx(2), 0]

bc1 = DirichletBC(W.sub(0), Constant(0.), boundaries, 1)
bc2 = DirichletBC(W.sub(1), Constant(0.), boundaries, 1)
bcs = BlockDirichletBC([bc1,
                        bc2,
                        None])

In [ ]:
A = block_assemble(a)
F = block_assemble(f)
bcs.apply(A)
bcs.apply(F)

U = BlockFunction(W)
block_solve(A, U.block_vector(), F)

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
A_ex = assemble(inner(grad(u), grad(v)) * dx)
F_ex = assemble(v * dx)
bc_ex = DirichletBC(V, Constant(0.), boundaries, 1)
bc_ex.apply(A_ex)
bc_ex.apply(F_ex)
U_ex = Function(V)
solve(A_ex, U_ex.vector(), F_ex)

In [ ]:
import numpy as np

In [ ]:
err1 = Function(V)
err1.vector().add_local(+ U_ex.vector().get_local())
err1.vector().add_local(- U[0].vector().get_local())
err1.vector().apply("")
err2 = Function(V)
err2.vector().add_local(+ U_ex.vector().get_local())
err2.vector().add_local(- U[1].vector().get_local())
err2.vector().apply("")
err1_norm = sqrt(assemble(err1 * err1 * dx(1)) / assemble(U_ex * U_ex * dx(1)))
err2_norm = sqrt(assemble(err2 * err2 * dx(2)) / assemble(U_ex * U_ex * dx(2)))
print("Relative error on subdomain 1", err1_norm)
print("Relative error on subdomain 2", err2_norm)
assert np.isclose(err1_norm, 0., atol=1.e-10)
assert np.isclose(err2_norm, 0., atol=1.e-10)

In [ ]:
%%bash

export LD_PRELOAD=""
ERROR_LIBRARIES=($(find /root/.cache/dijitso -name '*\.so' -exec \
    bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
    echo "No reference to libstdc++.so was found"
else
    for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
        echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
        ldd -v $ERROR_LIBRARY
    done
    false
fi